In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc
from sklearn.preprocessing import LabelEncoder

In [38]:
def get_train_test(complete_df):
    #split, process and return  X_train, X_test, y_train, y_test
    df_train, df_test = train_test_split(complete_df, test_size=0.3,stratify=complete_df['LABEL'])
    
    count = df_train['LABEL'].value_counts()
    count[count.isin([min(count)])].index[0]
    under_sampled = df_train[df_train['LABEL']==count[count.isin([max(count)])].index[0]].sample(min(count))
    balanced_df = pd.concat([under_sampled, df_train[df_train['LABEL']==count[count.isin([min(count)])].index[0]]], axis=0)
    X_train = balanced_df[balanced_df.columns[0:-1]]
    X_test = df_test[balanced_df.columns[0:-1]]
    y_label = balanced_df['LABEL']
    y_label_test = df_test['LABEL']
    le = LabelEncoder()
    le.fit(y_label)
    # le.classes_
    y_train = le.transform(y_label)
    y_test = le.transform(y_label_test)
    return X_train, X_test, y_train, y_test


In [63]:
complete_df= pd.read_csv('Data/New_features.csv')
elise_df = pd.read_csv('Data/additional_data_elise.csv')
youss_df = pd.read_csv('Data/Extra_features.csv')
elise_df = elise_df[elise_df.columns[1:]]
elise_df['LABEL'] = 'REAL'
andrea_features = pd.read_csv('Data/Andrea_features.csv')
full_df = pd.concat([complete_df,elise_df,youss_df])
X_train, X_test, y_train, y_test = get_train_test(full_df)
andrea_features.LABEL = andrea_features.LABEL.map({'n.wav' : 1, 'ouss_15min.w' : 0})
X_train = pd.concat([X_train,andrea_features[andrea_features.columns[0:-1]]])
y_train = pd.concat([pd.DataFrame(y_train),andrea_features[andrea_features.columns[-1]]])
np.array(y_train[0].T)

array([0, 0, 0, ..., 0, 0, 0])

In [64]:

# Average CV score on the training set was: 0.744065028547899
exported_pipeline = make_pipeline(
    RobustScaler(),
    KNeighborsClassifier(n_neighbors=44, p=1, weights="uniform")
)

exported_pipeline.fit(X_train, np.array(y_train[0].T))
results = exported_pipeline.predict(X_test)

In [65]:
accuracy_score(y_test,results)

0.943687628702677

In [69]:
precision_score(y_test,results)

0.9314893617021277

In [70]:
def test_model(svm_XGB_best2):
    #test model with our tests
    
    #read tests
    new_test_taylor_r = pd.read_csv('Data/extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.csv')
    new_test_bidenf = pd.read_csv('Data/audio_features_biden_AI.csv')
    new_test_bidenr = pd.read_csv('Data/audio_features_biden.csv')
    new_test_el = pd.read_csv('Data/Elise.csv')
    new_test_yr1 = pd.read_csv('Data/Youssef.csv')
    new_test_yr2 = pd.read_csv('Data/Youssef_2.csv')
    new_test_yf1 = pd.read_csv('Data/Youssef_Eric Cartman.csv')
    new_test_yf2 = pd.read_csv('Data/Youssef_2_Female.csv')
    new_test_yf3  = pd.read_csv('Data/Youssef_2_Male Reggaeton.csv')
    new_test_MF_f  = pd.read_csv('Data/Morgan Freeman_fake.csv')
    new_test_MF_r  = pd.read_csv('Data/Morgan Freeman_real.csv')
    new_test_And_r = pd.read_csv('Data/Andrea.csv')
    new_test_And_f = pd.read_csv('Data/Andrea_to_Youss.csv')
    fake_yous_n1 = pd.read_csv('Data/Youssef converted.csv')
    fake_yous_and2 = pd.read_csv('Data/Matt_Abrahams_to_Youss_f.csv')
    fake_yous_n2 = pd.read_csv('Data/Youssef_converted_model_1.csv')
    fake_yous_n3 = pd.read_csv('Data/Youssef_converted_model_2_crepe_v2.csv')
    fake_yous_n4 = pd.read_csv('Data/Youssef_converted_model_2_crepe.csv')
    fake_yous_and3 = pd.read_csv('Data/andie_to_youss2.csv')
    real_and2 = pd.read_csv('Data/andie_cats.csv')

    
    #make y_true
    y_real = np.ones((1000,))
    y_fake = np.zeros((1000,))
    
    #predict and get accuray score
    pred = svm_XGB_best2.predict(new_test_taylor_r)
    taylor_r = accuracy_score(y_real[:len(pred)],pred)
    print('Real Taylor : ' , taylor_r)
    
    pred = svm_XGB_best2.predict(new_test_bidenf[new_test_bidenf.columns[0:-2]])
    bidenf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Biden : ' , bidenf)

    pred = svm_XGB_best2.predict(new_test_bidenr[new_test_bidenf.columns[0:-2]])
    bidenr=accuracy_score(y_real[:len(pred)],pred)
    print('Real Biden : ' , bidenr)

    pred = svm_XGB_best2.predict(new_test_el)
    eliser=accuracy_score(y_real[:len(pred)],pred)
    print('Real Elise : ' , eliser)

    pred = svm_XGB_best2.predict(new_test_yr1)
    youssr1 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_1 : ' , youssr1)

    pred = svm_XGB_best2.predict(new_test_yr2)
    youssr2 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_2 : ' , youssr2)

    pred = svm_XGB_best2.predict(new_test_yf1)
    youssf1 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_1 : ' , youssf1)

    pred = svm_XGB_best2.predict(new_test_yf2)
    youssf2 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_2 : ' , youssf2)

    pred = svm_XGB_best2.predict(new_test_yf3)
    youssf3 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_3 : ' , youssf3)

    pred = svm_XGB_best2.predict(new_test_MF_f)
    morganf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Morgan : ' , morganf)

    pred = svm_XGB_best2.predict(new_test_MF_r)
    morganr = accuracy_score(y_real[:len(pred)],pred)
    print('Real Morgan : ' , morganr)

    pred = svm_XGB_best2.predict(new_test_And_f)
    andf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Andrea : ' , andf)

    pred = svm_XGB_best2.predict(new_test_And_r)
    andr = accuracy_score(y_real[:len(pred)],pred)
    print('Real Andrea : ' , andr)

    pred = svm_XGB_best2.predict(fake_yous_and2)
    f_youss_and = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake youss_and : ' , f_youss_and)

    pred = svm_XGB_best2.predict(fake_yous_n1)
    f_youss_n1 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_n1 : ' , f_youss_n1)

    pred = svm_XGB_best2.predict(fake_yous_n2)
    f_youss_n2 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_n2 : ' , f_youss_n2)

    pred = svm_XGB_best2.predict(fake_yous_n3)
    f_youss_n3 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_n3 : ' , f_youss_n3)

    pred = svm_XGB_best2.predict(fake_yous_n4)
    f_youss_n4 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_n4 : ' , f_youss_n4)

    pred = svm_XGB_best2.predict(fake_yous_and3)
    fake_yous_and3 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_and3 : ' , fake_yous_and3)

    pred = svm_XGB_best2.predict(real_and2)
    real_and2 = accuracy_score(y_real[:len(pred)],pred)
    print('Real_and2 : ' , real_and2)
    
    
    

In [71]:
test_model(exported_pipeline)

Real Taylor :  0.4
Fake Biden :  0.8
Real Biden :  0.9222222222222223
Real Elise :  0.043478260869565216
Real Youss_1 :  0.25
Real Youss_2 :  0.9166666666666666
Fake Youss_1 :  0.5
Fake Youss_2 :  0.6666666666666666
Fake Youss_3 :  0.08333333333333333
Fake Morgan :  0.5873015873015873
Real Morgan :  0.7241379310344828
Fake Andrea :  1.0
Real Andrea :  0.9230769230769231
Fake youss_and :  0.9829787234042553
Fake Youss_n1 :  0.9322033898305084
Fake Youss_n2 :  0.4745762711864407
Fake Youss_n3 :  0.6779661016949152
Fake Youss_n4 :  0.6101694915254238
Fake Youss_and3 :  1.0
Real_and2 :  0.8571428571428571
